In [21]:
import warnings 
warnings.filterwarnings(action='ignore')

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import json
import time

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [5]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

In [6]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

In [7]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

In [8]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

In [9]:
y_train.value_counts()

악플여부
0       55644
1       23944
dtype: int64

In [10]:
from imblearn.over_sampling import SMOTE

In [11]:
start = time.time()

smote = SMOTE(random_state=123)
train_over, y_train_over = smote.fit_sample(train, y_train)

print("걸린시간 :", time.time() - start)

걸린시간 : 226.63439559936523


In [ ]:
# train_over = train
# y_train_over = y_train

In [14]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [16]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 300
comment_len = 400

In [47]:
def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D, GlobalAveragePooling1D, Flatten, Bidirectional, LSTM, AveragePooling1D

    model = Sequential()
    model.add(Embedding(vocab_size+1, embedding_dim, input_length=comment_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(30, 3, padding='same', activation='relu', strides=1))
    model.add(AveragePooling1D())
    model.add(Bidirectional(LSTM(30)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.003), metrics=['accuracy'])

    return model


In [48]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights=True)
mc = ModelCheckpoint('1D_CNN_best.h5', monitor='val_loss', mode='min', save_best_only=True)

In [49]:
model = get_model()
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 50,
                    batch_size       = 32, 
                    validation_data = (test, y_test))

Epoch 1/50
3478/3478 [==============================] - 109s 31ms/step - loss: 0.3481 - accuracy: 0.8508 - val_loss: 0.2755 - val_accuracy: 0.8922
Epoch 2/50
3478/3478 [==============================] - 107s 31ms/step - loss: 0.2346 - accuracy: 0.9069 - val_loss: 0.2544 - val_accuracy: 0.9015
Epoch 3/50
3478/3478 [==============================] - 106s 30ms/step - loss: 0.2025 - accuracy: 0.9201 - val_loss: 0.2620 - val_accuracy: 0.8985
Epoch 4/50
3478/3478 [==============================] - 106s 30ms/step - loss: 0.1856 - accuracy: 0.9274 - val_loss: 0.2439 - val_accuracy: 0.9062
Epoch 5/50
3478/3478 [==============================] - 106s 31ms/step - loss: 0.1761 - accuracy: 0.9312 - val_loss: 0.2522 - val_accuracy: 0.8990
Epoch 6/50
3478/3478 [==============================] - 110s 32ms/step - loss: 0.1720 - accuracy: 0.9330 - val_loss: 0.2463 - val_accuracy: 0.9082
Epoch 7/50
3478/3478 [==============================] - 109s 31ms/step - loss: 0.1627 - accuracy: 0.9364 - val_loss: 0

In [50]:
# 저장한 CNN 모델을 불러오고 예측해보자


loaded_model = load_model('1D_CNN_best.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5

In [51]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score


print('recall        : ', recall_score(y_class, y_test))
print('precision     : ', precision_score(y_class, y_test))
print('f1_score      : ', f1_score(y_class, y_test))
print('accuracy      :', accuracy_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))
print(confusion_matrix(y_class, y_test))

recall        :  0.8516697745380573
precision     :  0.8351063829787234
f1_score      :  0.843306756189677
accuracy      : 0.9061667588078605
roc_auc_score :  0.9512182507430178
[[13006   992]
 [  875  5024]]


In [33]:
compare_df = pd.DataFrame({
    '댓글' : np.array(X_test).reshape(1, -1)[0],
    '예측' : y_class.reshape(1, -1)[0],
    '실제' : y_test.values.reshape(1, -1)[0]
})

compare_df['예측'] = compare_df['예측'].apply(int)

In [34]:
compare_df[compare_df['예측'] != compare_df['실제']].to_csv('/content/drive/MyDrive/[final_project]_악플원정대/no_match.csv')

In [35]:
def model_test(comment):
    loaded_model = load_model('1D_CNN_best.h5')
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [46]:
comment = '''옥상에서 떨어지세요'''

result, prob = model_test(comment)

악플이 아닙니다
라벨확률 :  [[0.22985938]]
